In [2]:
from cpselect.cpselect import cpselect
import skimage as sk 
import skimage.io as skio
import json
from scipy.spatial import Delaunay
import matplotlib.pyplot as plt
import numpy as np
from skimage.draw import polygon
from skimage.draw import polygon2mask
from skimage.transform import resize
import imageio
import os

In [3]:
img2 = skio.imread("original/rose_1.jpg")
skio.imshow(img2)

FileNotFoundError: No such file: '/Users/decemiles/cs194/proj3/FaceMorpher/original/rose_1.jpg'

In [4]:
img1 = skio.imread("original/haohan_1.jpg")
skio.imshow(img1)

FileNotFoundError: No such file: '/Users/decemiles/cs194/proj3/FaceMorpher/original/haohan_1.jpg'

In [5]:

def selPoints(img1, img2, file): 
    controlpointlist = cpselect(img1, img2)
    with open(file, 'w') as f:
        json.dump(controlpointlist, f)

# selPoints("original/haohan_1.jpg", "original/rose_1.jpg", 'points_1.json')

In [6]:
def triStack(triangle): 
    triangle = triangle[:, (1, 0)]
    row1 = np.hstack((triangle[0], [1]))
    row1New = np.hstack((row1 ,[0, 0, 0]))
    row2 = np.hstack(([0, 0, 0], row1))
    row3 = np.hstack((triangle[1], [1]))
    row3New = np.hstack((row3 ,[0, 0, 0]))
    row4 = np.hstack(([0, 0, 0], row3))
    row5 = np.hstack((triangle[2], [1]))
    row5New = np.hstack((row5 ,[0, 0, 0]))
    row6 = np.hstack(([0, 0, 0], row5))
    result = np.vstack((row1New, row2, row3New, row4, row5New, row6))
    return result 

def triStack1(triangle): 
    triangle = triangle[:, (1, 0)]
    a = np.hstack((triangle[0], triangle[1], triangle[2]))
    return a

def computeAffine(tri1_pts, tri2_pts):
    matrix1 = triStack(tri1_pts)
    matrix2 = triStack1(tri2_pts)
    result = np.linalg.solve(matrix1, matrix2)
    result = result.reshape(2, 3)
    result = np.vstack((result, [0, 0, 1]))
    result = np.linalg.inv(result)
    return result 

def MidwayTris(tri1, tri2, warp_frac=0.5): 
    return (1 - warp_frac) * tri1 + warp_frac * tri2

def preProcessPoints(file): 
    img1 = []
    img2 = []
    with open(file, 'r') as f: 
        datastore = json.load(f)
    
    for point in datastore: 
        temp1 = [point['img1_x'], point['img1_y']]
        temp2 = [point['img2_x'], point['img2_y']]
        img1.append(temp1)
        img2.append(temp2)
    img1 = np.array(img1)
    img2 = np.array(img2)
    return img1, img2

def crossDissolve(img1, img2, dis_frac=0.5): 
    return (1 - dis_frac) * img1 + dis_frac * img2
    
    

In [7]:
img1Co, img2Co = preProcessPoints('points_1.json')
midCo = MidwayTris(img1Co, img2Co)
tri = Delaunay(midCo)

FileNotFoundError: [Errno 2] No such file or directory: 'points_1.json'

In [8]:
plt.rcParams['figure.figsize'] = (20,10)
plt.triplot(img2Co[:,0], img2Co[:,1], tri.simplices)
plt.plot(img2Co[:,0], img2Co[:,1], 'o')
plt.imshow(img2)
plt.show()

NameError: name 'img2Co' is not defined

In [9]:


def morph(img1, img2, img1Co, img2Co, tri, warp_frac=0.5, dissolve_frac=0.5):  
    haohanTri = img1Co[tri.simplices.copy()]
    roseTri = img2Co[tri.simplices.copy()]
    resultTri = MidwayTris(haohanTri, roseTri, warp_frac)
    morphImg1 = np.zeros(img1.shape)
    morphImg2 = np.zeros(img2.shape)

    height, width, _ = img2.shape

    for x in range(haohanTri.shape[0]):
        
        A = computeAffine(haohanTri[x], resultTri[x])
        B = computeAffine(roseTri[x], resultTri[x])
        x_index = resultTri[x][:, 0]
        y_index = resultTri[x][:, 1]    
        rr, cc = polygon(y_index, x_index)

        test = np.vstack((rr, cc, [1 for x in range(len(rr))]))

        result1 = np.array(np.dot(A, test), dtype=int)
        result2 = np.array(np.dot(B, test), dtype=int)
        morphImg1[np.clip(test[0,:], 0, height -1), np.clip(test[1,:], 0, width-1), :] = \
            img1[np.clip(result1[0,:], 0, height -1), np.clip(result1[1,:], 0, width-1), :]
        morphImg2[np.clip(test[0,:], 0, height -1), np.clip(test[1,:], 0, width-1), :] = \
            img2[np.clip(result2[0,:], 0, height -1), np.clip(result2[1,:], 0, width-1), :]
    
    morphImg1 = morphImg1/255.0
    morphImg2 = morphImg2/255.0
    morphImg = crossDissolve(morphImg1, morphImg2, dissolve_frac)
     
    return morphImg
    
img1Co, img2Co = preProcessPoints('points_1.json')

midCo = MidwayTris(img1Co, img2Co)
tri = Delaunay(midCo)
midWayFace = morph(img1, img2, img1Co, img2Co, tri, 0.5, 0.5)
skio.imsave("midwayface.jpg", midWayFace)
skio.imshow(midWayFace) 


    

FileNotFoundError: [Errno 2] No such file or directory: 'points_1.json'

In [10]:
def generateGif(img1, img2, img1Co, img2Co, tri, gifname): 
    images = []
    for frac in np.linspace(0, 1, 45): 
        morphedImg = morph(img1, img2, img1Co, img2Co, tri, frac, frac)
        images.append(resize(morphedImg, (morphedImg.shape[0]//4, morphedImg.shape[1]//4)))
    imageio.mimsave(gifname, images, duration=1/30)

#generateGif(img1, img2, img1Co, img2Co, tri, 'hh2rosesmall.gif')

In [11]:
def preProcessPointsImm(imName):
    absPos = './data/imm_face_db/' + imName
    
    img = skio.imread(absPos)
    width, height, _ = img.shape
    absAsfPos = './data/imm_face_db/' + imName.split('.')[0] + '.asf'
    
    with open(absAsfPos, 'r') as f: 
        data = f.read()
    data.split('\n')
    
    xyIndex = data.split('\n')[16:74]
    points = [[row.split('\t')[2],row.split('\t')[3]] for row in xyIndex]
    result = [[float(point[0]) * height, float(point[1]) * width]for point in points]
    result.extend([[0,0],[width-1,0],[0,height-1],[width-1,height-1]])
    return img, np.array(result)

def morph1Img(img1, img1Co, meanCo, tri):  
    haohanTri = img1Co[tri.simplices.copy()]
    resultTri = meanCo[tri.simplices.copy()]
    morphImg1 = np.zeros(img1.shape)
    width, height, _ = img1.shape

    for x in range(haohanTri.shape[0]):    
        A = computeAffine(haohanTri[x], resultTri[x])
        x_index = resultTri[x][:, 0]
        y_index = resultTri[x][:, 1]    
        rr, cc = polygon(y_index, x_index)

        test = np.vstack((rr, cc, [1 for x in range(len(rr))]))
        test = np.array(test, dtype=int)
        result1 = np.array(np.dot(A, test), dtype=int)
        morphImg1[np.clip(test[0,:], 0, width -1), np.clip(test[1,:], 0, height-1), :] = \
            img1[np.clip(result1[0,:], 0, width -1), np.clip(result1[1,:], 0, height-1), :]
    
     
    return morphImg1



imnames = os.listdir('./data/imm_face_db')
imnames = [f for f in imnames if f.endswith('1f.jpg') or f.endswith('1m.jpg')]

imgs = []
immPoints = []
for imname in imnames: 
    img, points = preProcessPointsImm(imname)
    imgs.append(img)
    immPoints.append(points)

meanShape = np.mean(immPoints, axis=0)
immTri = Delaunay(meanShape)


morphedImgs = []

for i in range(len(imgs)): 
        temp = morph1Img(imgs[i], immPoints[i], meanShape, immTri)
        morphedImgs.append(temp)

avgface = np.mean(np.array(morphedImgs), axis=0)
avgface = avgface / 255.0
skio.imsave('danishAvg.jpg', avgface)
skio.imshow(avgface)
skio.imsave('avgMorphed1.jpg', morphedImgs[10])
skio.imsave('avgMorphed2.jpg', morphedImgs[8])
skio.imsave('avgMorphed3.jpg', morphedImgs[35])

FileNotFoundError: [Errno 2] No such file or directory: './data/imm_face_db'

In [12]:
selPoints("./danishAvgv1.jpg", "./haohanv1.jpg", "danish_me_co.json")

FileNotFoundError: [Errno 2] No such file or directory: './danishAvgv1.jpg'

In [13]:
meImg = skio.imread("./haohanv1.jpg")
danishImg = skio.imread("./danishAvgv1.jpg")
danish, me = preProcessPoints('danish_me_co.json')
danishTri = Delaunay(danish)
meTri = Delaunay(me)
danishMorph = morph(meImg, danishImg, me, danish, danishTri, 1, 0)
meMorph = morph(danishImg, meImg, danish, me, meTri, 1, 0)
skio.imshow(meMorph)
skio.imsave('me2Danish.jpg', danishMorph)
skio.imsave('danish2Me.jpg', meMorph)

FileNotFoundError: No such file: '/Users/decemiles/cs194/proj3/FaceMorpher/haohanv1.jpg'

In [14]:
danishMorph1 = morph(meImg, danishImg, me, danish, danishTri, 1, 0)
danishMorph2 = (danishMorph1 * 255.0) * 0.3 + danishImg * 0.7
danishMorph2 = danishMorph2 / 255.0

skio.imsave("danishMe.jpg", danishMorph2)
skio.imshow(danishMorph2)

NameError: name 'meImg' is not defined